In [1]:
#%load get_bookinfo.py
#%run get_bookinfo.py
# -*- coding: utf-8 -*-
#________________________________________________
import os
import django
from django.utils import timezone
from django.core.cache import cache
#from django.utils.dateparse import parse_datetime
from datetime import datetime,date#,timezone
import pytz
from bs4 import BeautifulSoup
from bs4.element import NavigableString
from difflib import SequenceMatcher
from IPython.display import clear_output, display
from time import sleep, time
from threading import Thread
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor 

from fake_useragent import UserAgent
from fake_headers import Headers
from pyquery import PyQuery as pq

#
import requests
import pandas as pd
import numpy as np
import random
import re
import json
import csv

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'wtb.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

#https://free-proxy-list.net/
def get_proxy(which='free',now=False,sample=False,sampleN=0):
    which_dict={'kuai':"/home/pan/django_projects/wtb/ips_kuai.txt",
                'ihuan':"/home/pan/django_projects/wtb/ips_1000_ihuan.txt",
                'free':"/home/pan/django_projects/wtb/ips_free.txt",
                'OK':"/home/pan/django_projects/wtb/ips_OKs.txt"
               }
    #
    ippos=cache.get('ippos')
    if not ippos:
        ippos=[]
        with open(which_dict[which], 'r') as f:
            lines=f.readlines()
            for line in lines:
                line=line.strip() #remove space_\t_\n
                if len(line.split(":"))==2:
                    ippos.append(line)
        #
        cache.set('ippos', ippos, 86400)    
    #
    if sample and sampleN>0:        
        return random.sample(ippos,sampleN)
    if now:
        s=0.5+random.uniform(0, 1)
        sleep(s)
        return random.choice(ippos)
    #ippo
    old=[]
    #測試代理_________________________________________
    while True:
        fake_header = Headers(
            browser="chrome",  # Generate only Chrome UA
            os="win",  # Generate ony Windows platform
            headers=True  # generate misc headers
        )    
        UA=fake_header.generate()
        #UA = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        #已經測過的就不要
        ippo=random.choice(ippos)
        if len(old)/len(ippos) > 0.5:
            old=[]
        if ippo in old:
            continue
        else:
            old.append(ippo)
        #    
        ip=ippo.split(":")[0]
        proxies={
                "http": "http://"+ippo,
                "https": "http://"+ippo
                }  
        #print("test:"+ippo)    
        #clear_output(wait=True)
        try:
            test=['http://icanhazip.com/','https://myip.com.tw/','https://www.showmyipaddress.eu/']
            r = requests.get(random.choice(test),#test[1], 
                             headers=UA,
                             proxies=proxies,
                             timeout=4)
            r.encoding='utf8'
            #print(r.status_code)
            #print(r.text)
            #return
            #return 22
            #
            if(r.status_code == 200 and (ip in r.text or '&#46;'.join(ip.split('.')) in r.text) ):
                print("OK:"+which+"_"+ippo)
                #print(r.status_code)
                #print(r.text)
                r.close() 
                if which!='OK':
                    fn='/home/pan/django_projects/wtb/ips_OKs.txt'
                    mode=0                    
                    if mode==0:
                        #加到最後一行
                        with open(fn, 'a+') as f:
                            f.write(ippo+"\r\n")      
                    else:
                        #刪掉第一行_加到最後一行
                        with open(fn, 'r+') as f: # open file in read / write mode
                            firstLine = f.readline() # read the first line and throw it out
                            data = f.read() # read the rest
                            data += ippo+"\r\n"
                            f.seek(0) # set the cursor to the top of the file
                            f.write(data) # write the data back
                            f.truncate() # set the file size to the current size 
                #______________
                return ippo
            else:#if not (r.status_code == 200 and ip in r.text):
                r.close()
                raise Exception('fail')
               
        except Exception as e:
            #print(str(e))
            #clear_output(wait=True) 
             
            sleep(1.5+random.uniform(0, 2))
            #sleep(0.1)
            continue
        #else:
        #    print(r.text)
        #    print("OK2:"+ip)
        #    r.close()       
        #    break    


In [ ]:
if __name__ == '__main__':
    which_dict={
                #'kuai':"ips_kuai.txt",
                #'ihuan':"ips_1000_ihuan.txt",
                'free':'ips_free.txt',
                #'OK':'ips_OKs.txt'
               }
    for i in range(100):
        print(i)
        with ThreadPoolExecutor(max_workers=3) as executor:
            executor.map(get_proxy,which_dict.keys())

0
OK:free_110.39.187.50:49850
1
OK:free_193.41.88.58:53281
2
OK:free_151.253.165.70:8080
3
OK:free_128.199.237.62:44344
4
OK:free_196.3.97.71:23500
5
OK:free_54.37.14.65:3128
6
OK:free_36.67.57.45:53367
7
OK:free_176.101.89.226:33470
8
OK:free_187.177.139.8:8080
9
OK:free_186.56.99.103:53040
10
OK:free_173.217.255.36:33351
11
OK:free_91.211.107.204:41258
12
OK:free_167.172.188.118:3128
13
OK:free_110.78.148.37:41521
14
OK:free_101.255.103.201:53281
15
OK:free_36.89.192.119:52313
16
OK:free_190.18.207.58:53281
17
OK:free_124.41.243.22:47894
18
OK:free_128.199.246.10:44344
19
OK:free_202.5.221.66:80
20
OK:free_128.199.150.150:47503
21
OK:free_103.94.171.134:46276
22
OK:free_110.74.200.246:45755
23


In [ ]:
#kuai,ihuan,free
#%time get_proxy('OK',True,sample=True,sampleN=3)
AAA()
#print(a)

In [ ]:
with open('/home/pan/django_projects/wtb/ips_OKs.txt', 'r') as f:
    lines=f.readlines()
    for l in lines:
        print(l.strip()+"_")
        #print(l+"__")

In [ ]:
ippos=[]
with open('/home/pan/django_projects/wtb/ips_OKs.txt', 'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip() #remove space_\t_\n
        if len(line.split(":"))==2:
            ippos.append(line)

In [ ]:
1+2